In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from scipy.stats import chi2_contingency

import requests
from bs4 import BeautifulSoup

In [2]:
%run '0_tools.ipynb'

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [4]:
stopWords = ['and', 'at', 'the', 'yet', 'so', 'because', 'on', 'of', 'to', 'as',' in', 'his', 'her', 'she', 'him',
            'them', 'they', 'it', 'hers', 'their', 'its', 'theirs', 'with', 'said', 'for', 'after', 'will', 'that',
            'about', 'who', 'by', 'all', 'where', 'over', 'year', 'years', 'continue', 'two', 'three', 'four', 'five',
            'six', 'seven', 'eight', 'nine', 'ten', 'other', 'into']

In [5]:
def trimWord(word):
    w = word
    
    if ((w[len(w)-1] == ",")or(w[len(w)-1] == ".")or(w[len(w)-1] == "?")
        or(w[len(w)-1] == "!")or(w[len(w)-1] == "’")or(w[len(w)-1] == ":")
        or(w[len(w)-1]=="\"")or(w[len(w)-1]=="-")or(w[len(w)-1]=="'")):
        w = w[:len(w)-1]
                        
    if((w[0]=="'")or(w[0]=="’")or(w[0]=="\"")or(w[0]=="-")):
        w = w[1:]
                        
    if ((w[len(w)-2:len(w)] == "’s")or(w[len(w)-2:len(w)] == "\'s")):
        w = w[:len(w)-2]
    
    return w

In [6]:
def scrapeArticlesCNN(w, rBank, dBank):
    dataFile = open(f"{data_path}political_articles/cnn.txt", "a")
    
    www = w
    www = requests.get(www,headers = headers)

    if (www.status_code != 200):
        return "Error"

    soup = BeautifulSoup(www.text, 'html.parser')    
    
    headlineBlocks = soup.find_all('span', class_ = 'sitemap-link')
    headlineBlocks = headlineBlocks[1:]
    #for i in headlineBlocks:
    #    print(i)
    x=0
    for currentHeadline in headlineBlocks:
        x+=1
        articleWWW = str(currentHeadline.find('a')).split("\"")[1]
        #print(articleWWW)
        articleWWW = requests.get(articleWWW,headers = headers)
        
        if (www.status_code != 200):
            return "Error"
        
        soup = BeautifulSoup(articleWWW.text, 'html.parser')
        
        
        #-------------------------------------------------------------------------------------------------------------
        headline = soup.find('h1', class_="headline__text inline-placeholder")
        #print(headline.text.split())
        
        r = False
        d = False
        try:
            headline_words = headline.text.split()
        except:
            continue
            
            
        for word in headline_words:
            try:
                word = trimWord(word)
                word = word.lower()
            except:
                continue
                
            if(word in stopWords):
                continue
            
            if (not r):
                r = (word in rBank)
            if (not d):
                d = (word in dBank)
            
            
        if (not ((r or d) & (not (r&d)))):
            continue
        #------------------------------------------------------------------------------------------------------------- 
        paragraph = soup.find_all('p', class_ = 'paragraph inline-placeholder')
        
        try:
            if (r):
                dataFile.write(paragraph[0].text + paragraph[1].text + paragraph[2].text + ' r     \n')
            if (d):
                dataFile.write(paragraph[0].text + paragraph[1].text + paragraph[2].text + ' d     \n')
        except:
            continue
        

    dataFile.close()
    
    return "done"

In [26]:
www = "https://www.multistate.us/resources/2023-governors-and-legislatures"

www = requests.get(www,headers = headers)
if (www.status_code != 200):
    print("Error")

soup = BeautifulSoup(www.text, 'html.parser')

politicians = []
parties = []

govBlocks = soup.find_all("td", class_ = "gov left")

for govBlock in govBlocks:
    
    vect = govBlock.text.split()
    governor = vect[1]
    party = vect[len(vect)-1][1]
    
    if(party=="D"):
        party = "Democratic"
    if(party=="R"):
        party = "Republican"
            
    politicians.append(governor.lower())
    parties.append(party)

www = "https://ballotpedia.org/List_of_current_members_of_the_U.S._Congress"

www = requests.get(www,headers = headers)
if (www.status_code != 200):
    print("Error")

soup = BeautifulSoup(www.text, 'html.parser')
        
people = soup.find_all('tr')

for i in people:
    try:
        name = i.find('td', style="padding-left:10px;text-align:center;").text
        party = ""
        try:
            party = i.find("td", class_ = "partytd Democratic").text
            party = "Democratic"
        except:
            party = i.find("td", class_ = "partytd Republican").text
            party = "Republican"
        
        names = name.split()
        last_name = names[len(names)-1]
        politicians.append(last_name.lower())
        parties.append(party)
        
    except:
        continue

In [56]:
additionalRepublicans = ['trump', 'g.o.p.', 'gop', 'republicans', 'republican', 'conservatives', 'pence', 'herschel']
additionalDemocrats = ['pelosi', 'biden', 'democrat', 'democrats', 'liberals', 'kamala', 'obama', 'bernie', 'clinton']

In [57]:
people = ""
pdf = pd.DataFrame()
pdf['Politician'] = politicians
pdf['Party'] = parties

for a in additionalRepublicans:
    pdf.loc[len(pdf['Party'].values)] = a.lower()
    pdf['Party'].loc[len(pdf['Party'].values)-1] = 'Republican'

for b in additionalDemocrats:
    pdf.loc[len(pdf['Party'].values)] = b.lower()
    pdf['Party'].loc[len(pdf['Party'].values)-1] = 'Democratic'

In [58]:
#remove some politicians with generic names
removePoliticians = ['justice', 'cooper', 'cox', 'murphy', 'green', 'greene', 'little', 'moore', 'reeves', 'shapiro', 
                     'lee', 'scott', 'jr.', 'miller', 'rodgers', 'good', 'fitzgerald', 'smith', 'tiffany', 'carson', 
                    'stevens', 'james', 'harris', 'davis', 'carter', 'clark', 'jackson', 'collins', 'bush', 'thompson'
                    'gordon', 'young', 'paul', 'peters', 'brown', 'murray', 'iii', 'johnson' 'carl' 'rogers', 'strong',
                    'crane', 'hill', 'kim', 'steel', 'crow', 'courtney', 'bean', 'frost', 'posey', 'wilson', 'williams',
                    'allen', 'simpson', 'flood', 'ryan', 'ross', 'adams', 'edwards', 'jordan', 'lucas', 'cole', 'rose',
                    'fallon', 'roy', 'drew', 'clarke', 'wild', 'gordon', 'kennedy', 'schmitt', 'booker', 'vance', 
                     'mullin', 'reed', 'graham', 'evans', 'dean', 'duncan', 'case', 'hunt', 'golden', 'carey', 'joyce',
                    'johnson','graves', 'davidson', 'franklin', 'cassidy', 'garcia', 'mills', 'thompson', 'curtis',
                    'stewart', 'guest', 'cloud', 'banks', 'self', 'waters', 'buck', 'game', 'foster', 'rounds', 'carl',
                    'kiley', 'mast', 'rogers', 'owens', 'fry', 'norman', 'crenshaw']

for r in removePoliticians:
    try:
        pdf = pdf.drop(pdf.index[pdf['Politician'] == r.lower()].tolist())
        
    except:
        continue

In [59]:
republicans = list(pdf[pdf['Party'] == 'Republican']['Politician'])
democrats = list(pdf[pdf['Party'] == 'Democratic']['Politician'])

In [61]:
# np.save(f'{data_path}political_articles/republicans.npy', np.array(republicans))
# np.save(f'{data_path}political_articles/democrats.npy', np.array(democrats))

In [62]:
republicans = list(np.load(f'{data_path}political_articles/republicans.npy'))
democrats = list(np.load(f'{data_path}political_articles/democrats.npy'))

In [63]:
startDate = '10/2022'
endDate = '04/2023'

In [15]:
startDate = startDate.split('/')
endDate = endDate.split('/')

In [16]:
startMonth = int(startDate[0])
startYear = int(startDate[1])

endMonth = int(endDate[0])
endYear = int(endDate[1])

In [17]:
months = ['no', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 
          'October', 'November', 'December']

In [18]:
def daysInMonth(m, y):
    if(m==2): 
        if (y%100==0):
            if(y%400==0): return 29
            else: return 28
        elif(y%4==0): return 29
        else: return 28
    elif ((m==1)or(m==3)or(m==5)or(m==7)or(m==8)or(m==10)or(m==12)): return 31
    else: return 30

In [19]:
# open(f'{data_path}political_articles/cnn.txt', 'w').close()

# for year in range(startYear, endYear+1):
    
#     if (year == startYear):
#         beginM = startMonth
#     else:
#         beginM = 1
        
#     if (year == endYear):
#         endM = endMonth
#     else:
#         endM = 12
    
#     for month in range(beginM,endM+1):
#         print(f"{months[month]} {year}")
#         print(scrapeArticlesCNN(f"https://www.cnn.com/politics/article/sitemap-{str(year)}-{str(month)}.html", republicans, democrats))

In [24]:
# def scrapeArticlesWSJ(w, rBank, dBank):
#     dataFile = open(f"{data_path}political_articles/WSJ.txt", "a")
    
#     print(w, '\n')
    
#     www = w
#     www = requests.get(www,headers = headers)

#     if (www.status_code != 200):
#         return "Error"

#     soup = BeautifulSoup(www.text, 'html.parser')    
    
#     articleBlocks = soup.find_all('article')
    
#     for articleBlock in articleBlocks:
#         r=False
#         d=False
#         bloc = articleBlock.find('a')
# #         print(bloc.find('span').text)
#         bloc = (str(bloc).split("\""))
    
#         articleWWW = 'Error'
        
#         for b in bloc:
#             if (b[:8]=="https://"): 
#                 articleWWW = b
#                 break
                
#         if (articleWWW == 'Error'): continue
                
#         headlineWords = (articleWWW[29:]).split('-')
#         for word in headlineWords:
#             if (word in rBank): r=True
#             if (word in dBank): d=True
#             if (r&d): break
                
#         if (not ((r or d) & (not (r&d)))): continue
         
#  #     ____________________________________________________________________________________________________
         
        
#         print(articleWWW)
        
#         articleWWW = requests.get(articleWWW,headers = headers)

#         if (articleWWW.status_code != 200):
#             return "Error"

#         articleSoup = BeautifulSoup(articleWWW.text, 'html.parser')    
#         print(articleSoup.text)
        
#         break
#  #     ____________________________________________________________________________________________________
                
#     dataFile.close()

In [25]:
# startDate = '10/2022'
# endDate = '11/2022'

# startDate = startDate.split('/')
# endDate = endDate.split('/')

# startMonth = int(startDate[0])
# startYear = int(startDate[1])

# endMonth = int(endDate[0])
# endYear = int(endDate[1])

# open(f'{data_path}political_articles/WSJ.txt', 'w').close()

# for year in range(startYear, endYear+1):
    
#     if (year == startYear):
#         beginM = startMonth
#     else: beginM = 1
        
#     if (year == endYear):
#         endM = endMonth
#     else: endM = 12

#     for month in range(beginM, endM+1):
#         endDay = daysInMonth(month, year)
#         for day in range(1, endDay+1):
#             if (day < 10): d = '0' + str(day)
#             else: d = str(day)
#             scrapeArticlesWSJ(f'https://www.wsj.com/news/archive/{str(year)}/{str(month)}/{d}', republicans, democrats)
            
#             break
#         break
#     break

In [103]:
def scrapeWashingtonExaminer(w, rBank, dBank):
    dataFile = open(f"{data_path}political_articles/WashExam.txt", "a")
    
#     print(w, '\n')
    
    www = w
    www = requests.get(www,headers = headers)

    if (www.status_code != 200):
        return "Error"

    soup = BeautifulSoup(www.text, 'html.parser')    
    
    articleBlocks = soup.find_all('li', class_='ArchivePage-items-item link-item')
    
    links = []
    
    for articleBlock in articleBlocks:
        headline = (str(articleBlock.text))
        r=False
        d=False
        for i in headline.split():
            word = i.lower()
            if (word in rBank): r=True
            if (word in dBank): d=True
            if (r&d): break
                
#         if(r&d): print(headline)
        if (not ((r or d) & (not (r&d)))): continue
            
        articleWWW = articleBlock.find('a')['href']
        
        if(articleWWW.find(' ')!=-1): continue
                    
                
        articleWWW = requests.get(articleWWW,headers = headers)

        if (articleWWW.status_code != 200):
            return "Error"

        articleSoup = BeautifulSoup(articleWWW.text, 'html.parser')    
        try:
            text_blocks = (articleSoup.find('div', class_='RichTextArticleBody-body')).find_all('p')
        except: continue
            
        num_t = 0
        finalText = ''
        for i in text_blocks: 
            if(num_t >= 3): break
            text = i.text
            if (text.isupper()): continue
        
            finalText += text
            num_t+=1
            
        if (r): dataFile.write(finalText + ' r     \n')
        elif(d): dataFile.write(finalText + ' d     \n')        
    
    
    dataFile.close()
    

In [104]:
startDate = '10/2022'
endDate = '04/2023'

startDate = startDate.split('/')
endDate = endDate.split('/')

startMonth = int(startDate[0])
startYear = int(startDate[1])

endMonth = int(endDate[0])
endYear = int(endDate[1])

open(f'{data_path}political_articles/WashExam.txt', 'w').close()

for year in range(startYear, endYear+1):
    
    if (year == startYear):
        beginM = startMonth
    else: beginM = 1
        
    if (year == endYear):
        endM = endMonth
    else: endM = 12

    for month in range(beginM, endM+1):
        print(f"{months[month]} {year}", end = '')
        endDay = daysInMonth(month, year)
        
        for day in range(1, endDay+1):
            scrapeWashingtonExaminer(f'https://www.washingtonexaminer.com/sitemap/{str(year)}/{str(month)}/{str(day)}', republicans, democrats)         
            
        print('done.')

print('All done.')

October 2022
November 2022
December 2022
January 2023
February 2023
March 2023
April 2023
